In [1]:
%%file mapper_1.py
#!/usr/bin/env python

# import sys
import sys
first = True

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    if (first):
        first=False
    else:    
        # remove leading and trailing whitespace
        line = line.strip()
        # split the line into words
        words = line.split(',')
        # words[24] = state_name
        print(words[24]+'_'+str(words[5])+str(words[6]),'\t','1')

Writing mapper_1.py


In [2]:
%%file reducer_1.py
#!/usr/bin/python

import sys

prev_state=""
pp_state=""
prev_lat_long=""
pp_lat_long=""
first = True
total_count=0

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    state_lat_long, count = line.split('\t')
    state,lat_long = line.split('_')
    if first:
        total_count+=int(count)
        pp_state=prev_state
        prev_state=state
        pp_lat_long=prev_lat_long
        prev_lat_long=lat_long
        first=False
    else:
        if prev_state == state and lat_long!=prev_lat_long:
            total_count+=int(count)
            pp_state=prev_state
            prev_state=state  
            pp_lat_long=prev_lat_long
            prev_lat_long=lat_long
        if lat_long != prev_lat_long:
            pp_lat_long=prev_lat_long
            prev_lat_long=lat_long
        if prev_state != state:
            print('%s\t%s' % (prev_state,total_count))  
            total_count = int(count)
            pp_state=prev_state
            prev_state=state                
if pp_state == state:
    print('%s\t%s' % (prev_state, total_count))

Writing reducer_1.py


In [3]:
%%file mapper_1_2.py
#!/usr/bin/python

import sys

for line in sys.stdin:
    line = line.strip()
    state, count = line.split('\t')
#    count_z_filled=str(count).zfill(3)
    count_999=999-int(count)
#    print('%s\t%s' % (count_z_filled,state))
    print('%s\t%s' % (count_999,state))

Writing mapper_1_2.py


In [4]:
%%file reducer_1_2.py
#!/usr/bin/python

import sys


for line in sys.stdin:

    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    count, state = line.split('\t')
    count_999=999-int(count)
    print('%s %s' % (state,count_999))

Writing reducer_1_2.py


In [5]:
rm -rf results

In [6]:
rm -rf results_1_2

In [7]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_1.py,reducer_1.py -mapper mapper_1.py -reducer reducer_1.py -input epa_hap_daily_summary-small.csv -output results
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_1_2.py,reducer_1_2.py -mapper mapper_1_2.py -reducer reducer_1_2.py -input results -output results_1_2

2021-12-19 21:04:43,200 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2021-12-19 21:04:43,262 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-12-19 21:04:43,262 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-12-19 21:04:43,276 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-12-19 21:04:43,364 INFO mapred.FileInputFormat: Total input files to process : 1
2021-12-19 21:04:43,372 INFO mapreduce.JobSubmitter: number of splits:4
2021-12-19 21:04:43,472 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local119549087_0001
2021-12-19 21:04:43,472 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-12-19 21:04:43,592 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_1.py as file:/tmp/hadoop-jovyan/mapred/local/job_local119549087_0001_95ab4994-6f07-41b0-96ea-21f943daf357/mapper_1.py
2021-12-19 21:04:43,598 INFO mapred.LocalDistr

In [8]:
!cat results_1_2/part-*

California 170	
Texas 133	
Minnesota 94	
Michigan 92	
Ohio 91	
New York 67	
South Carolina 64	
Montana 62	
Pennsylvania 61	
Florida 55	
Indiana 52	
Colorado 51	
North Carolina 50	
Illinois 49	
Washington 43	
Louisiana 41	
Arizona 38	
Kansas 37	
Georgia 35	
Kentucky 34	
Oregon 32	
Alabama 31	
Tennessee 29	
Wisconsin 26	
New Jersey 24	
Vermont 23	
Oklahoma 22	
Mississippi 21	
Maine 21	
Virginia 19	
Massachusetts 19	
Missouri 18	
New Mexico 18	
Iowa 18	
Country Of Mexico 18	
New Hampshire 17	
Maryland 17	
Idaho 17	
Connecticut 15	
Rhode Island 13	
Alaska 12	
Utah 12	
Arkansas 11	
West Virginia 10	
Nevada 9	
Wyoming 9	
North Dakota 7	
South Dakota 7	
Nebraska 6	
Puerto Rico 6	
Delaware 6	
Virgin Islands 6	
Hawaii 5	
District Of Columbia 5	


In [9]:
%%file mapper_2.py
#!/usr/bin/env python

# import sys
import sys

first = True

# input comes from STDIN (standard input)
for line in sys.stdin:
    if (first):
        first=False
    else:    
        # remove leading and trailing whitespace
        line = line.strip()
        # split the line into words
        words = line.split(',')
        # words[24] = state_name
#        if float(words[16]) > 0: 
        print('%s\t%s' % (words[25],words[16]))

Writing mapper_2.py


In [10]:
%%file reducer_2.py
#!/usr/bin/env python

# import sys
import sys

pp_counti=""
prev_counti=""
first=True
avg_total=0
n=0
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    counti, avg_value = line.split('\t')

    if counti == prev_counti : #verifica se o counti ainda é o mesmo
        avg_total+=float(avg_value) #somatório das médias
        n+=1
    else:
        if first:
            first = False
            prev_counti=counti 
        else: #caso seja o primeiro não executa pq o prev_counti não existe
            avg=avg_total/n
            print('%s\t%.15f' % (prev_counti,avg))

            pp_counti=prev_counti
            prev_counti=counti 
            avg_total=0
            n=0

        avg_total+=float(avg_value) #somatório das médias para quando é a priemeira vez do counti
        n+=1      

if counti == pp_counti:  # caso o ultimo state fosse igual ao anterior ele nao dava print
    avg=avg_total/n
    print('%s\t%.15f' % (prev_counti,avg))
    ##########################
    #os resultados dão zeros#
    #########################

Writing reducer_2.py


In [11]:
%%file mapper_2_1.py
#!/usr/bin/python

import sys

for line in sys.stdin:
    line = line.strip()
    state, count = line.split('\t')
#    count_z_filled=str(count).zfill(3)
    count_999=999-float(count)
#    print('%s\t%s' % (count_z_filled,state))
    print('%s\t%s' % (count_999,state))

Writing mapper_2_1.py


In [12]:
%%file reducer_2_1.py
#!/usr/bin/python

import sys


for line in sys.stdin:

    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    count, state = line.split('\t')
    count_999=999-float(count)
    print('%s %s' % (state,count_999))

Writing reducer_2_1.py


In [13]:
rm -rf results_2

In [14]:
rm -rf results_2_2

In [15]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_2.py,reducer_2.py -mapper mapper_2.py -reducer reducer_2.py -input epa_hap_daily_summary-small.csv -output results_2
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_2_1.py,reducer_2_1.py -mapper mapper_2_1.py -reducer reducer_2_1.py  -input results_2/part-* -output results_2_2

2021-12-19 21:12:04,780 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2021-12-19 21:12:04,813 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-12-19 21:12:04,814 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-12-19 21:12:04,823 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-12-19 21:12:04,898 INFO mapred.FileInputFormat: Total input files to process : 1
2021-12-19 21:12:04,906 INFO mapreduce.JobSubmitter: number of splits:4
2021-12-19 21:12:04,972 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1647055560_0001
2021-12-19 21:12:04,972 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-12-19 21:12:05,071 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_2.py as file:/tmp/hadoop-jovyan/mapred/local/job_local1647055560_0001_9ae1e250-95a5-4f38-8070-e0fc53842649/mapper_2.py
2021-12-19 21:12:05,077 INFO mapred.LocalDis

In [16]:
!cat results_2_2/part-*

Tipton 2556.0	
Nassau 19.0	
Columbiana 7.385690736	
CHIHUAHUA STATE 4.5121875	
Caldwell 4.116666667	
Madera 3.7393	
Oakland 2.888877848	
Duval 2.779460398	
Kearny 2.375333333	
Bucks 2.3675	
San Luis Obispo 2.333333333	
Edgecombe 2.325	
Pawnee 2.294117647	
Westchester 2.239375	
Johnston 2.225	
Hartford 2.07870559	
Granville 2.028571429	
Asotin 2.025	
Duplin 2.0	
Boulder 1.960470901	
Yancey 1.9	
Crittenden 1.9	
Los Angeles 1.839135086	
Iberville 1.821313227	
Caswell 1.80075	
Pitt 1.8	
Clinton 1.758301887	
Wayne 1.713796421	
Imperial 1.647860052	
Ozaukee 1.55733332	
Stillwater 1.538461538	
Crow Wing 1.532046512	
Boyd 1.485702255	
Gloucester 1.481428571	
Henderson 1.429550909	
Kennebec 1.375	
Stanislaus 1.278194127	
Muscatine 1.2375	
Deer Lodge 1.213579339	
Mesa 1.210952492	
Spokane 1.146973351	
East Baton Rouge 1.144472846	
Harris 1.142705829	
Davis 1.141848124	
Phillips 1.141409396	
Tuscola 1.136961406	
Nez Perce 1.12264928	
Bayamon 1.120036364	
Pipestone 1.080746341	
Wyandotte 1.0779660

In [63]:
%%file mapper_3.py
#!/usr/bin/env python

# import sys
import sys

first = True

# input comes from STDIN (standard input)
for line in sys.stdin:
    if (first):
        first=False
    else:    
        # remove leading and trailing whitespace
        line = line.strip()
        # split the line into words
        words = line.split(',')
        year=words[11][0:4]
        # words[24] = state_name
#        if float(words[16]) != 0:
        print(year+words[24],'\t',words[16])

Overwriting mapper_3.py


In [64]:
%%file reducer_3.py
#!/usr/bin/env python


# import sys
import sys

pp_state=""
prev_state=""
prev_year=""
first=True
avg_total=0
n=0
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    year_plus_state, avg_value = line.split('\t')
    
    state=year_plus_state[4:] #divisao do ano do estado
    year=year_plus_state[0:4]
    if state == prev_state : #verifica se o counti ainda é o mesmo
        avg_total+=float(avg_value) #somatório das médias
        n+=1
    else:
        if first:
            n=+1
            prev_state=state
            prev_year=year
            first = False
        else: #caso seja o primeiro não executa pq o prev_state não existe
            avg=avg_total/n
            print('%s\t%s\t%.15f' % (prev_year,prev_state,avg))
            avg_total=0
            n=0        
            pp_state=prev_state
            prev_state=state
            prev_year=year
            
        avg_total+=float(avg_value) #somatório das médias para quando é a priemeira vez do state
        n+=1
        
        if state==pp_state:
            avg=avg_total/n
            print('%s\t%s\t%.15f' % (prev_year,prev_state,avg))

Writing reducer_3.py


In [65]:
%%file mapper_3_1.py
#!/usr/bin/python

import sys

for line in sys.stdin:
    line = line.strip()
    year, counti, count = line.split('\t')
    count_z_filled=str(count).zfill(20)
    print(year+count_z_filled+'\t'+counti)

Overwriting mapper_3_1.py


In [66]:
%%file reducer_3_1.py
#!/usr/bin/python

import sys


for line in sys.stdin:

    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    year_plus_count, counti = line.split('\t')
    year=year_plus_count[0:4] #divisao do ano do estado
    count=year_plus_count[4:]
    print('%s\t%s\t%s' % (int(year), counti,float(count)))

Overwriting reducer_3_1.py


In [67]:
rm -rf results_3

In [68]:
rm -rf results_3_1

In [69]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_3.py,reducer_3.py -mapper mapper_3.py -reducer reducer_3.py -input epa_hap_daily_summary-small.csv -output results_3
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_3_1.py,reducer_3_1.py -mapper mapper_3_1.py -reducer reducer_3_1.py -input results_3/part-* -output results_3_1

2021-12-19 21:21:32,607 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2021-12-19 21:21:32,640 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-12-19 21:21:32,640 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-12-19 21:21:32,649 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-12-19 21:21:32,718 INFO mapred.FileInputFormat: Total input files to process : 1
2021-12-19 21:21:32,726 INFO mapreduce.JobSubmitter: number of splits:4
2021-12-19 21:21:32,793 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1516839329_0001
2021-12-19 21:21:32,793 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-12-19 21:21:32,902 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_3.py as file:/tmp/hadoop-jovyan/mapred/local/job_local1516839329_0001_9c975918-b30b-4cce-afd5-a4dfbb33d24d/mapper_3.py
2021-12-19 21:21:32,908 INFO mapred.LocalDis

In [70]:
!cat results_3_1/part-*

1990	Wisconsin	0.0
1990	Oklahoma	0.0
1990	Virgin Islands	0.0
1990	West Virginia	0.0
1990	Hawaii	0.000197037037037
1990	Nevada	0.0004208
1990	Alaska	0.000442083333333
1990	South Dakota	0.0005705
1990	Washington	0.0005975
1990	Wyoming	0.000604545454545
1990	Utah	0.000797058823529
1990	New Mexico	0.000822222222222
1990	Oregon	0.00085962962963
1990	Arizona	0.000862013422819
1990	Maine	0.000978928571429
1990	Colorado	0.00216237410072
1990	Mississippi	0.00266666666667
1990	Missouri	0.0056
1990	Michigan	0.00655989637306
1990	Connecticut	0.0081
1990	Georgia	0.00836666666667
1990	Puerto Rico	0.01005
1990	North Carolina	0.0143
1990	Alabama	0.01946
1990	Iowa	0.0332
1990	Pennsylvania	0.1059625
1990	Ohio	0.135716
1990	Illinois	0.145757012195
1990	New Jersey	0.293335294118
1990	Minnesota	0.306488
1990	California	0.411530998363
1990	South Carolina	0.559814035088
1990	District Of Columbia	0.826150819672
1990	Virginia	0.845141666667
1990	Louisiana	0.914594594595
1990	Kansas	1.14081545741
1990	Florida	1

In [39]:
%%file mapper_4.py
#!/usr/bin/env python

# import sys
import sys

first = True
dic={}
with open('usa_states.csv') as file_extra:
    for line2 in file_extra:
        if (first):
            first=False
        else:
            line2 = line2.strip()
            # split the line into words
            words_2 = line2.split(',')            
            dic.update({words_2[1] : (words_2[2],words_2[3],words_2[4],words_2[5])})
first = True
# input comes from STDIN (standard input)
for line in sys.stdin:
    if (first):
        first=False
    else:
        line = line.strip()
        # split the line into words
        words = line.split(',')
        # words[24] = state_name
        if dic.get(words[24],"0") != "0":# and (words[5]!= '0' and words[6]!= '0'):
            print('%s_%s_%s\t%s\t%s\t%s\t%s' % (words[24],words[5],words[6],str(dic.get(words[24])[0]),str(dic.get(words[24])[1]),str(dic.get(words[24])[2]),str(dic.get(words[24])[3])))

Writing mapper_4.py


In [40]:
%%file reducer_4.py
#!/usr/bin/env python

# import sys
import sys
import math
prev_words=""
prev_state=""
avg_total=0
n=0
first=True
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    words= line.split('\t')
    if words!=prev_words:
        state, lat, long= words[0].split('_')
        if state != prev_state and first==False:
            avg=avg_total/n
            print('%s\t%.15f' % (prev_state,avg))
            avg_total=0
            n=0
        center_lat=(float(words[1])+float(words[2]))/2
        center_long=(float(words[3])+float(words[4]))/2
        dist=math.sqrt(((center_lat-float(lat))*111)**2+((center_long-float(long))*111)**2)
        avg_total+=dist #somatório das distancias
        n+=1
    first=False
    prev_words=words
    prev_state=state
if state == prev_state:
    avg=avg_total/n
    print('%s\t%.15f' % (prev_state,avg))

Writing reducer_4.py


In [41]:
%%file mapper_4_1.py
#!/usr/bin/python

import sys

for line in sys.stdin:
    line = line.strip()
    state, count = line.split('\t')
#    count_z_filled=str(count).zfill(3)
    count_999=9999-float(count)
#    print('%s\t%s' % (count_z_filled,state))
    print('%.15f\t%s' % (count_999,state))
#    print('%.5f\t%s' % (float(count),state))

Writing mapper_4_1.py


In [42]:
%%file reducer_4_1.py
#!/usr/bin/python

import sys


for line in sys.stdin:

    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    count, state = line.split('\t')
    count_999=9999-float(count)
    print('%s %.15f' % (state,count_999))

Writing reducer_4_1.py


In [43]:
rm -rf results_4

In [44]:
rm -rf results_4_1

In [45]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_4.py,reducer_4.py -mapper mapper_4.py -reducer reducer_4.py -input epa_hap_daily_summary-small.csv -output results_4
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_4_1.py,reducer_4_1.py -mapper mapper_4_1.py -reducer reducer_4_1.py -input results_4/part-* -output results_4_1

2021-12-19 21:16:09,141 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2021-12-19 21:16:09,172 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-12-19 21:16:09,172 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-12-19 21:16:09,181 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-12-19 21:16:09,252 INFO mapred.FileInputFormat: Total input files to process : 1
2021-12-19 21:16:09,261 INFO mapreduce.JobSubmitter: number of splits:4
2021-12-19 21:16:09,326 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1565049905_0001
2021-12-19 21:16:09,326 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-12-19 21:16:09,429 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_4.py as file:/tmp/hadoop-jovyan/mapred/local/job_local1565049905_0001_55f52509-1086-4637-b9f3-7c4eea8829a2/mapper_4.py
2021-12-19 21:16:09,435 INFO mapred.LocalDis

In [55]:
!cat results_4_1/part-*

Virginia 715.432973026043328	
Alaska 603.699642241068432	
Texas 512.183989163013393	
Vermont 504.063235485080440	
Illinois 440.854014321286741	
South Dakota 365.845137432326737	
Florida 336.544914657082700	
California 328.226381315536855	
Michigan 326.411606485113225	
Nevada 326.281180719739496	
Nebraska 307.141182605528229	
Kansas 292.079684129670568	
Idaho 289.635073275630930	
Montana 286.838335275693680	
New York 283.727339863717134	
Wyoming 283.640586337474815	
Oregon 268.853807923263957	
Pennsylvania 251.415176340571634	
North Dakota 248.421930732627516	
Oklahoma 236.882574372981253	
Tennessee 235.976149514875942	
Missouri 234.329534127940860	
Washington 219.980448067997713	
Kentucky 219.951516807521330	
Iowa 206.598941036891119	
Wisconsin 202.846930079784215	
Minnesota 195.068270828247478	
Utah 184.913430407115811	
Georgia 184.348235073470278	
New Mexico 183.189121285236979	
Colorado 180.259742033151269	
North Carolina 179.094048153569020	
Arizona 178.875740556120945	
Indiana 177

In [47]:
%%file mapper_5.py
#!/usr/bin/env python

# import sys
import sys

first = True
dic={}
with open('usa_states.csv') as file_extra:
    for line2 in file_extra:
        if (first):
            first=False
        else:
            line2 = line2.strip()
            # split the line into words
            words_2 = line2.split(',')            
            dic.update({words_2[1] : (words_2[2],words_2[3],words_2[4],words_2[5])})
first = True
# input comes from STDIN (standard input)
for line in sys.stdin:
    if (first):
        first=False
    else:
        line = line.strip()
        # split the line into words
        words = line.split(',')
        # words[24] = state_name
        if dic.get(words[24],"0") != "0": #and (words[5]!= '0' and words[6]!= '0'):
            print('%s_%s_%s\t%s\t%s\t%s\t%s' % (words[24],words[5],words[6],str(dic.get(words[24])[0]),str(dic.get(words[24])[1]),str(dic.get(words[24])[2]),str(dic.get(words[24])[3])))

Writing mapper_5.py


In [48]:
%%file reducer_5.py
#!/usr/bin/env python

import sys
import math
prev_words=""
prev_state=""
NW=0
NE=0
SE=0
SW=0
first=True
# input comes from STDIN (standard input)
print('state, (NE,SE,SW,NW)')
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    words= line.split('\t')
    if words!=prev_words:
        state, lat, long= words[0].split('_')
        if state != prev_state and first==False:
            print('%s\t%d\t%d\t%d\t%d' % (prev_state,NE,SE,SW,NW))
            NW=0
            NE=0
            SE=0
            SW=0
        center_lat=(float(words[1])+float(words[2]))/2
        center_long=(float(words[3])+float(words[4]))/2
        if float(lat)<center_lat and float(long)>center_long:
            SE+=1
        if float(lat)>center_lat and float(long)>center_long:
            NE+=1
        if float(lat)<center_lat and float(long)<center_long:
            SW+=1
        if float(lat)>center_lat and float(long)<center_long:
            NW+=1
    first=False
    prev_words=words
    prev_state=state
if state == prev_state:
            print('%s\t%d\t%d\t%d\t%d' % (prev_state,NE,SE,SW,NW))

Writing reducer_5.py


In [49]:
%%file mapper_5_1.py
#!/usr/bin/python

import sys

for line in sys.stdin:
    line = line.strip()
    state, count = line.split('\t')
#    count_z_filled=str(count).zfill(3)
    count_999=9999-float(count)
#    print('%s\t%s' % (count_z_filled,state))
    print('%.15f\t%s' % (count_999,state))
#    print('%.5f\t%s' % (float(count),state))

Writing mapper_5_1.py


In [50]:
%%file reducer_5_1.py
#!/usr/bin/python

import sys


for line in sys.stdin:

    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    count, state = line.split('\t')
    count_999=9999-float(count)
    print('%s %.15f' % (state,count_999))

Writing reducer_5_1.py


In [51]:
rm -rf results_5

In [52]:
rm -rf results_5_1

In [53]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_5.py,reducer_5.py -mapper mapper_5.py -reducer reducer_5.py -input epa_hap_daily_summary-small.csv -output results_5

2021-12-19 21:17:27,947 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2021-12-19 21:17:27,980 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-12-19 21:17:27,980 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-12-19 21:17:27,990 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-12-19 21:17:28,062 INFO mapred.FileInputFormat: Total input files to process : 1
2021-12-19 21:17:28,070 INFO mapreduce.JobSubmitter: number of splits:4
2021-12-19 21:17:28,133 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local685299330_0001
2021-12-19 21:17:28,133 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-12-19 21:17:28,234 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_5.py as file:/tmp/hadoop-jovyan/mapred/local/job_local685299330_0001_f3ae6658-a6fc-4ec0-aa35-d848733538c0/mapper_5.py
2021-12-19 21:17:28,239 INFO mapred.LocalDistr

In [54]:
!cat results_5/part-*

state, (NE,SE,SW,NW)	
Alabama	5	5	7	14
Alaska	4	2	3	3
Arizona	2	16	10	10
Arkansas	2	1	5	3
California	2	68	16	84
Colorado	25	4	5	17
Connecticut	5	0	8	2
Delaware	0	0	2	4
Florida	27	23	0	5
Georgia	4	5	5	21
Hawaii	2	1	0	2
Idaho	0	3	7	7
Illinois	32	1	14	2
Indiana	18	8	8	18
Iowa	6	8	4	0
Kansas	18	9	8	2
Kentucky	13	2	16	3
Louisiana	0	35	1	5
Maine	2	6	12	1
Maryland	16	0	0	1
Massachusetts	11	4	0	4
Michigan	14	74	1	3
Minnesota	11	50	21	12
Mississippi	4	12	5	0
Missouri	9	3	2	4
Montana	11	5	36	10
Nebraska	1	3	0	2
Nevada	2	5	0	2
New Hampshire	2	11	4	0
New Jersey	18	1	4	1
New Mexico	1	2	3	12
New York	5	43	3	16
North Carolina	26	5	0	19
North Dakota	1	1	3	2
Ohio	30	10	36	15
Oklahoma	18	2	0	2
Oregon	3	1	13	15
Pennsylvania	3	36	18	4
Puerto Rico	4	0	1	1
Rhode Island	12	0	0	1
South Carolina	10	18	3	33
South Dakota	0	4	3	0
Tennessee	8	4	8	9
Texas	34	72	3	24
Utah	0	3	3	6
Vermont	1	1	5	16
Virgin Islands	0	0	6	0
Virginia	8	7	4	0
Washington	6	2	15	20
West Virginia	2	0	5	3
Wisconsin	2	21	1	2
Wyoming	2	2	2	3
